In [1]:
!pip install ensemble_boxes

In [28]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [29]:
import pandas as pd
import cv2
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [51]:
# ensemble csv files
#,'detectors_4842.csv'
#5103,5099,5150
#submission_files = ['seoyoon_scnet.csv','800800.csv','jin_SCNet_0.5150.csv']
submission_files = ['dd-softnms.csv','dd-nmw.csv','dd.csv']

submission_df = [pd.read_csv(file) for file in submission_files]

In [52]:
image_ids = submission_df[0]['image_id'].tolist()

In [53]:
annotation = '../../input/data/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [54]:
prediction_strings = []
file_names = []
iou_thr = 0.5
weights=[0.5,1,1]
sigma=0.1
skip_box_thr=0.0001
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        if len(predict_list)==0 or len(predict_list)==1:
            continue
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    if len(boxes_list):
        
#         boxes1, scores1, labels1 =  weighted_boxes_fusion(boxes_list, scores_list, labels_list,weights=weights, iou_thr=iou_thr)#skip_box_thr=0.0
        #boxes, scores, labels = soft_nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
        #boxes, scores, labels = non_maximum_weighted(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        #boxes4, scores4, labels4 = nms(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr)
        boxes, scores, labels =  weighted_boxes_fusion(boxes_list, scores_list, labels_list,weights=weights, iou_thr=iou_thr)#skip_box_thr=0.0
        
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:73: UserWarning: X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('X2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')
/opt/conda/lib/python3.7/site-packages/ensemble_boxes/ensemble_boxes_wbf.py:85: UserWarning: Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.
  warnings.warn('Y2 > 1 in box. Set it to 1. Check that you normalize boxes in [0, 1] range.')


In [55]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('dd-ensemble.csv')

submission.head()

,PredictionString,image_id
0,8 0.7222182 173.31605529785156 232.09939575195...,batch_01_vt/0021.jpg
1,8 0.7218626737594604 156.6174774169922 73.3720...,batch_01_vt/0028.jpg
2,1 0.7095751 70.38508605957031 161.789657592773...,batch_01_vt/0031.jpg
3,8 0.6936653 161.99835205078125 125.09317016601...,batch_01_vt/0032.jpg
4,8 0.7222116 147.71644592285156 0.0 355.2125244...,batch_01_vt/0070.jpg
